In [ ]:
%%capture
!wget https://www.dropbox.com/scl/fi/gwssm0f77en2nuiu34o5m/MSK_Price_train.csv?rlkey=vg1hhva3yaxsvo3zl1flgfhi9&dl=0
!wget https://www.dropbox.com/scl/fi/4fbkx0r16yjemr245f848/MSK_Price_valid.csv?rlkey=be7phwzd5d4py8kiqwdfiuw8u&dl=0

In [ ]:
pip install catboost

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np
import catboost
from catboost import CatBoostRegressor
from catboost import Pool
from catboost import cv

In [ ]:
df = pd.read_csv('MSK_Price_train.csv')
df_test = pd.read_csv('MSK_Price_valid.csv')

In [ ]:
df.head()

,Unnamed: 0,Price,Apartment type,Metro station,Minutes to metro,Region,Number of rooms,Area,Living area,Kitchen area,Floor,Number of floors,Renovation
0,0,59990000.0,Secondary,Серпуховская,10.0,Moscow,2.0,73.00,38.8,10.0,4.0,14,Without renovation
1,1,5250753.0,New building,Красногвардейская,11.0,Moscow region,2.0,35.68,22.7,5.0,11.0,20,Cosmetic
2,2,14500000.0,Secondary,Алтуфьево,9.0,Moscow,2.0,53.60,33.0,10.0,5.0,16,Designer
3,3,15799990.0,Secondary,Рассказовка,30.0,Moscow,2.0,54.00,30.3,9.8,14.0,17,European-style renovation
4,4,9300000.0,Secondary,Новоясеневская,17.0,Moscow,1.0,33.00,18.0,11.0,9.0,9,Cosmetic


In [ ]:
del df['Unnamed: 0']
del df_test['Unnamed: 0']

### Перевод столбца Metro station в lower

In [ ]:
df['Metro station'] = df['Metro station'].str.lower().str.strip()
df_test['Metro station'] = df_test['Metro station'].str.lower().str.strip()

### Факторизация

In [ ]:
columns_names = ['Apartment type', 'Metro station', 'Region', 'Renovation']
df[columns_names] = df[columns_names].apply(lambda x: pd.factorize(x)[0])
df_test[columns_names] = df_test[columns_names].apply(lambda x: pd.factorize(x)[0])

In [ ]:
df = df[(df['Price'] < 1000000000)]
df.reset_index(inplace=True)
del df['index']

df = df[(df['Price'] > 2300000)]
df.reset_index(inplace=True)
del df['index']

df = df[(df['Number of rooms'] < 7)]
df.reset_index(inplace=True)
del df['index']

df = df[(df['Floor'] < 40)]
df.reset_index(inplace=True)
del df['index']

df = df[(df['Number of floors'] < 60)]
df.reset_index(inplace=True)
del df['index']


### Добавление новых признаков

In [ ]:
df

,Price,Apartment type,Metro station,Minutes to metro,Region,Number of rooms,Area,Living area,Kitchen area,Floor,Number of floors,Renovation
0,59990000.0,0,0,10.0,0,2.0,73.00,38.8,10.0,4.0,14,0
1,5250753.0,1,1,11.0,1,2.0,35.68,22.7,5.0,11.0,20,1
2,14500000.0,0,2,9.0,0,2.0,53.60,33.0,10.0,5.0,16,2
3,15799990.0,0,3,30.0,0,2.0,54.00,30.3,9.8,14.0,17,3
4,9300000.0,0,4,17.0,0,1.0,33.00,18.0,11.0,9.0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...
21224,10525091.0,1,17,7.0,1,3.0,75.40,40.3,12.9,11.0,14,1
21225,2638290.0,1,37,13.0,1,0.0,23.82,14.2,8.6,8.0,16,1
21226,14452240.0,0,134,4.0,0,6.0,23.80,12.9,2.3,16.0,20,1
21227,221000000.0,0,53,8.0,0,4.0,240.00,40.0,26.5,14.0,14,2


In [ ]:
def minutes_metro_to_category(minutes):
  if  minutes < 10.0 : return 0
  elif minutes <= 30.0 : return 1
  else : return 2

In [ ]:
df['Distance to metro']=df['Minutes to metro'].apply(minutes_metro_to_category)
df_test['Distance to metro']=df_test['Minutes to metro'].apply(minutes_metro_to_category)

In [ ]:
del df['Minutes to metro']
del df_test['Minutes to metro']

In [ ]:
df.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [ ]:
# def floor_category_func(floor_category):
#   if  floor_category < 0.2 : return 0
#   elif floor_category <= 0.9 : return 1
#   else : return 2

In [ ]:
def living_area_cat(living_area):
  if  living_area < 0.5 : return 0
  elif living_area <= 0.9 : return 1
  else : return 2

In [ ]:
df['Living Area cat'] = (df['Living area']/df['Area']).apply(living_area_cat)
df_test['Living Area cat'] = (df_test['Living area']/df_test['Area']).apply(living_area_cat)

# df['Floor_category'] = (df['Floor']/df['Number of floors']).apply(floor_category_func)
# df_test['Floor_category'] = (df_test['Floor']/df_test['Number of floors']).apply(floor_category_func)

# del df['Floor']
# del df_test['Floor']
# del df['Number of floors']
# del df_test['Number of floors']

# del df['Area']
# del df_test['Area']
# df.reset_index(inplace=True, drop=True)
# df_test.reset_index(inplace=True, drop=True)

In [ ]:
# df['livArea/num_r'] = df['Living area'] / (df['Number of rooms'] + 1)
# df_test['livArea/num_r'] = df_test['Living area'] / (df_test['Number of rooms'] + 1)
# df['Apartment type for Region'] = df['Apartment type'] + df['Region']
# df_test['Apartment type for Region'] = df_test['Apartment type'] + df_test['Region']
# df_test['Ratio of floors'] = df_test['Floor'] / df_test['Number of floors']
# df['Ratio of floors'] = df['Floor'] / df['Number of floors']

In [ ]:
del df['Kitchen area']
del df['Living area']
# del df['Region']
del df['Metro station']
del df['Renovation']
df.reset_index(inplace=True, drop=True)

In [ ]:
del df_test['Kitchen area']
del df_test['Living area']
del df_test['Metro station']
# del df_test['Region']
del df_test['Renovation']
df_test.reset_index(inplace=True, drop=True)

### Настройка гиперпараметров

In [ ]:
df

,Price,Apartment type,Region,Number of rooms,Area,Floor,Number of floors,Distance to metro,Living Area cat
0,59990000.0,0,0,2.0,73.00,4.0,14,1,1
1,5250753.0,1,1,2.0,35.68,11.0,20,1,1
2,14500000.0,0,0,2.0,53.60,5.0,16,0,1
3,15799990.0,0,0,2.0,54.00,14.0,17,1,1
4,9300000.0,0,0,1.0,33.00,9.0,9,1,1
...,...,...,...,...,...,...,...,...,...
21224,10525091.0,1,1,3.0,75.40,11.0,14,0,1
21225,2638290.0,1,1,0.0,23.82,8.0,16,1,1
21226,14452240.0,0,0,6.0,23.80,16.0,20,0,1
21227,221000000.0,0,0,4.0,240.00,14.0,14,0,0


In [ ]:
df_test

,Price,Apartment type,Region,Number of rooms,Area,Floor,Number of floors,Distance to metro,Living Area cat
0,4000000.0,0,0,0.0,11.70,1.0,9,1,1
1,15970000.0,0,0,2.0,57.30,28.0,33,1,0
2,4500000.0,0,0,0.0,16.00,1.0,14,1,1
3,14500000.0,0,0,1.0,39.00,2.0,17,0,0
4,9413360.0,1,1,2.0,56.30,14.0,17,0,0
...,...,...,...,...,...,...,...,...,...
1171,341000000.0,0,0,6.0,227.82,3.0,12,0,0
1172,7900000.0,0,0,0.0,19.00,5.0,5,0,1
1173,5285510.0,1,1,0.0,19.70,12.0,17,1,0
1174,8022810.0,1,1,3.0,58.39,2.0,14,1,1


In [ ]:
# from matplotlib import pyplot as plt
# df['Number of floors'].plot(kind='hist', title='Number of floors')
# plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# from matplotlib import pyplot as plt
# df['Floor'].plot(kind='hist', figsize=(8, 4), title='Floor')
# plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# from matplotlib import pyplot as plt
# df.plot(kind='scatter', x='Number of rooms', y='Area', s=32, alpha=.8)
# plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# from matplotlib import pyplot as plt
# df['Price'].plot(kind='line', figsize=(8, 4), title='Price')
# plt.gca().spines[['top', 'right']].set_visible(False)

### Построение модели

In [ ]:
train_values = df['Price']
train_points = df.drop('Price', axis=1)
test_values = df_test['Price']
test_points = df_test.drop('Price', axis=1)

In [ ]:
xg_reg = xgb.XGBRegressor(
                          learning_rate = 0.04,
                          max_depth = 9,
                          alpha= 10,
                          n_estimators = 2000,
                      )
#eval_metric="mae", eval_set=[(train_points, train_values), (test_points, test_values)]
xg_reg.fit(train_points, train_values)
xgb_predict = xg_reg.predict(test_points)
print(mean_absolute_error(test_values, xgb_predict))

8105477.089073129


In [ ]:
# results = xg_reg.evals_result()
# epochs = len(results['validation_0']['mae'])
# x_axis = range(0, epochs)

# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# ax.plot(x_axis, results['validation_0']['mae'], label='Train MAE')
# ax.plot(x_axis, results['validation_1']['mae'], label='Test MAE')
# ax.legend()
# plt.ylabel('MAE')
# plt.title('XGBoost MAE')
# plt.show()

In [ ]:
from catboost import CatBoostRegressor

clf = CatBoostRegressor(
    # iterations=500,
    learning_rate=0.15,
    depth=9,
    # loss_function='MAE'
)
clf.fit(train_points, train_values)
test_predict_cat = clf.predict(test_points)
print(mean_absolute_error(test_values, test_predict_cat))

0:	learn: 65661087.2018241	total: 7.9ms	remaining: 7.89s
1:	learn: 59662841.2738702	total: 13.3ms	remaining: 6.65s
2:	learn: 54872577.8657371	total: 20.4ms	remaining: 6.77s
3:	learn: 51134644.9390982	total: 23.8ms	remaining: 5.92s
4:	learn: 47889955.9493036	total: 39.8ms	remaining: 7.91s
5:	learn: 45282428.8647463	total: 55.4ms	remaining: 9.17s
6:	learn: 43022047.4464882	total: 62.3ms	remaining: 8.84s
7:	learn: 41166635.4141346	total: 69.8ms	remaining: 8.66s
8:	learn: 39769054.8321113	total: 77ms	remaining: 8.48s
9:	learn: 38523584.9830170	total: 85.2ms	remaining: 8.43s
10:	learn: 37450464.7971518	total: 93ms	remaining: 8.36s
11:	learn: 36616828.7919852	total: 100ms	remaining: 8.25s
12:	learn: 35967268.3709916	total: 107ms	remaining: 8.11s
13:	learn: 35325997.4432661	total: 114ms	remaining: 8.01s
14:	learn: 34897317.7520535	total: 121ms	remaining: 7.93s
15:	learn: 34526990.8355580	total: 128ms	remaining: 7.87s
16:	learn: 34147406.8491339	total: 136ms	remaining: 7.85s
17:	learn: 3374592